## Word2Vec

El modelo más famoso de word embeddings. Inventado por [Tomas Mikolov en 2013 en Google](https://arxiv.org/pdf/1310.4546.pdf)

https://radimrehurek.com/gensim/models/word2vec.html

### Hyperparameters

Algunos de los hiperparámetros que deberemos tener en cuenta y configuraremos:

- size: dimensionalidad de las palabras vector
- window: ventana para obtener el contexto de cada palabra. Se mide en número de palabras máximo entre la palabra actual y la palabra a predecir
- min_count: frecuencia mínima de aparición de una palabra para que sea considerada en el entrenamiento
- sg: algoritmo escogido. 1 para Skip-Gram, 0 para CBOW
- hs: si = 1, se utiliza una softmax jerárquica. Si = 0, y negative != 0, se emplea negative sampling
- negative: si = 0, no se usa negative sampling. Si es > 0, se usará negative sampling. El valor indica el número de "palabras ruidosas" se incluirán (usual entre 5-20 para datasets pequeños, entre 2-5 para datasets grandes)

### Noción de contexto

<img src=https://docs.chainer.org/en/v4.0.0b2/_images/center_context_word.png width=450px>

## Atributos

- wv: word vectors, contiene el mapeo entre palabras y vectores (embeddings)
- vocabulary: vocabulario (o diccionario) del modelo


## Negative sampling

Cuando se trabaja en NLP el tamaño del vocabulario suele tener una cardinalidad enorme. Esto afecta a los modelos de lenguaje a la hora de predecir aquellas palabras que, aunque correctas, no son demasiado frecuentes.

Además, contextos muy comunes (como los que podrían ser aquellos en los que se encuentran muchas stop words) hacen que el entrenamiento sea lento. Se emplea, por tanto, para reducir la carga computacional al problema.

La solución que se propone - e implementa - Word2Vec es que cada palabra tenga una determinada probabilidad de ser eliminada del training set. Dicha probabilidad estará relacionada con la frecuencia de repetición de dicha palabra.


## Arquitecturas

Existen dos arquitecturas de este modelo: CBOW y Skip Gram.


#### CBOW (Continuous Bag of Words)

Durante el entrenamiento, el modelo tratará de **predecir la palabra actual** dado el contexto en el que se encuentre. La capa de entrada contendrá las palabras-contexto y la de salida será la palabra actual (o palabra a predecir). La capa intermedia tendrá una dimension igual al número de dimensiones en el que queremos representar la palabra actual a la salida.

<img src=https://miro.medium.com/max/1104/0*CCsrTAjN80MqswXG width=400px>


#### Skip Gram

Durante el entrenamiento, el modelo tratará de predecir **el contexto (palabras-contexto)** a una palabra dada. La capa de entrada contendrá la palabra actual y la de salida serán las palabras contexto. La capa intermedia es análoga a la presente en la arquitectura CBOW.

<img src=https://miro.medium.com/max/1280/0*Ta3qx5CQsrJloyCA.png width=300px>

### ¿Cuál es mejor?

En general, depende. CBOW, al haber sido entrenado para predecir una palabra dado un contexto, será algo mejor _rellenando huecos_, aunque eso puede significar que palabras correctas pero menos comunes no aparezcan como resultado algunas veces. Skip Gram, en cambio, debería ser mejor infiriendo relaciones más concretas en contextos similares (por ejemplo, "me gusta el color verde" y "me encanta el color azul", y la diferencia en la intensidad del sentimiento expresado).

Si atendemos a los comentarios de Mikolov:

- Skip-gram: funciona bien con conjuntos de datos pequeños, representando bien incluso palabras o frases extrañas (poco comunes)
- CBOW: entrenamiento varias veces más rápido, su performance mejor para aquellas palabras más frecuentes que el resto

# Word Embeddings

Los Word Embeddings son aquellas técnicas y modelos de lenguaje que permiten mapear palabras a vectores de valores continuos. Durante el entrenamiento de dichos vectores se buscará que capturen la información semántica de las palabras.

<img src=https://miro.medium.com/max/1280/1*OEmWDt4eztOcm5pr2QbxfA.png widt=500px>

Gracias a que los vectores tienen información sobre la semántica, mediante operaciones vectoriales podemos encontrar palabras (o documentos) que tienen un significado similar.

La idea principal de este tipo de modelos es que **palabras que aparecen en contextos similares tienen semánticas similares**. **Concepto de sustituibilidad**.

**Aquellas palabras que semánticamente son similares tendrán - idealmente - vectores-palabra cercanos entre sí.**

Existen multitud de modelos, en esta sesión veremos solo algunas.

In [ ]:
!pip install unzip
!unzip simpsons_dataset.csv.zip

## Palabras más similares

## Importamos las librerías

In [ ]:
!pip install gensim
#Ejercicio de aplicación con Word2Vec
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence


In [ ]:
import re  # Para Preprocesamiento
import pandas as pd
from time import time  # Tiempo de las operaciones
from collections import defaultdict  # Para Frecuencias de palabras

import spacy  # Para prepocesamiento
import logging  # Configuración de loggings para monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from gensim.models.phrases import Phrases, Phraser

## Lectura de datos

In [ ]:
df = pd.read_csv('./simpsons_dataset.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

In [ ]:

nlp = spacy.load("en_core_web_sm")
def cleaning(doc):
    # Lematizamos y removemos stopwords
    # doc necesita ser a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec usa las palabras de contexto para aprender a representar el vector de una palabra ,
    # si una sentencia tiene solo una o dos palabras ,
    # el beneficio para el training es muy pequeño
    if len(txt) > 2:
        return ' '.join(txt)

Quitar los caracteres no alfabéticos

In [ ]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])

Utilizamos el atributo .pipe() de spaCy para acelerar la velocidad del proceso de limpieza

In [ ]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Tiempo para limpiar todo: {} mins'.format(round((time() - t) / 60, 2)))

Organizamos el resultado en una DataFrame eliminando los valores missing y duplicados:

In [ ]:
df_clean_simpsons = pd.DataFrame({'clean': txt})
df_clean_simpsons = df_clean_simpsons.dropna().drop_duplicates()
df_clean_simpsons.shape

In [ ]:
df_clean_simpsons.to_csv('./df_clean_simpsons.csv')

## Entrenamos el modelo

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

Separamos el training del modelo en tres pasos:

Word2Vec():
- En el primer paso preparamos los parametros del modelo  

.build_vocab():
- En este paso se construye el vocabulario de una secuencia de sentencias y así inicializamos el modelo. Con los loggings, podemos ver el efecto el efecto de   min_count y sample  sobre el word corpus. Estos parametros, en particular sample, tienen una gran influencia sobre el performance del modelo.

.train():
- Finalmente, entrenamos el modelo.
El loggings aquí puede ser útil para ir monitoriando.

In [ ]:
cores = multiprocessing.cpu_count() # Contamos el número de cores en el ordenador
print (cores)

## Hyperparameters e Inicializamos los objetos Word2Vec

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=cores-1)

## Construímos el vocabulario

Word2Vec requiere construir una tabla del vocabulario (procesamos todas las palabras, filtramos y las contamos):

In [ ]:
t = time()
sent = [row.split() for row in df_clean_simpsons['clean']]
w2v_model.build_vocab(sent, progress_per=10000)

print('Tiempo para construir el vocabulario: {} mins'.format(round((time() - t) / 60, 2)))


In [ ]:
print('Vocabulario compuesto por {} palabras'.format(len(w2v_model.wv.key_to_index)))



## Entrenamos el modelo

Parametros del training:

* total_examples = int - cuenta las sentencias
* epochs = int - Número de iteraciones (epochs) sobre el corpus - [10, 20, 30]

In [ ]:
t = time()

w2v_model.train(sent, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

## Guardamos los modelos

In [ ]:
w2v_model.save('./w2v_model.pkl')



## Algunos resultados

* Similares:

Preguntamos a nuestros modelo encontrar la palabra más similar a los personajes más populares de the Simpsons!

In [ ]:
w2v_model.wv.most_similar(positive=["homer"])


In [ ]:
w2v_model.wv.most_similar(positive=["marge"])

In [ ]:
w2v_model.wv.most_similar(positive=["bart"])

In [ ]:
w2v_model.wv.similarity('maggie', 'baby')

In [ ]:
w2v_model.wv.similarity('bart', 'nelson')

In [ ]:
w2v_model.wv.doesnt_match(['jimbo', 'milhouse'])

In [ ]:
w2v_model.wv.doesnt_match(['homer', 'patty', 'selma'])

## Palabras fuera del vocabulario (OOV Words)

Los embeddings calculados a nivel de palabra no devolverán un vector para aquellos tokens que no hayan guardado en su vocabulario. Una vez que los vectores palabra han sido aprendidos, aquellas palabras que no han sido aprendidas durante el entrenamiento no tendrán representación.

In [ ]:
'asereje' in w2v_model.wv.key_to_index

In [ ]:
w2v_model.wv.most_similar('asereje')

Algunas estrategias para lidiar con palabras OOV:
- Asignar un vector que siga una distribución aleatoria uniforme. P. ej.:
`unk = np.random.uniform(-np.var(w2v.wv.vectors), np.var(w2v.wv.vectors), w2v.wv.vector_size)`
- Reemplazar por un token especial, conocido y distinto del resto, (`<unk>`) y entrenar los embeddings
- Reemplazar por un token especial, conocido y disinto del resto, y añadir información extra. P. ej.: `<unk_noun>` o `<unk_verb>`
- Utilizar modelos que no sean a nivel de palabra

## Visualización: Bonus

[Enlace](https://anvaka.github.io/pm/#/galaxy/word2vec-wiki?cx=-16179&cy=-1641&cz=4313&lx=0.3194&ly=-0.5230&lz=-0.4110&lw=0.6749&ml=300&s=1.75&l=1&v=d50_clean)

<img src=https://empresas.blogthinkbig.com/wp-content/uploads/2019/06/embeddings_galaxy.png width=650px>